In [14]:
import spacy
import numpy
import os
import pathlib
from keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout
from keras.layers import TimeDistributed
from keras.models import Sequential, model_from_json
from keras.optimizers import Adam
from spacy.compat import pickle

pt_br_model = 'pt_wikipedia_md'

class SentimentAnalyser(object):
    @classmethod
    def load(cls, path, nlp, max_length=100):
        """
        Loads the language model file.
        """
        with (path / 'config.json').open() as file_:
            model = model_from_json(file_.read())
        with (path / 'model').open('rb') as file_:
            lstm_weights = pickle.load(file_)
        embeddings = get_embeddings(nlp.vocab)
        model.set_weights([embeddings] + lstm_weights)
        return cls(model, max_length=max_length)

    def __init__(self, model, max_length=100):
        self._model = model
        self.max_length = max_length

    def __call__(self, doc):
        X = get_features([doc], self.max_length)
        y = self._model.predict(X)
        self.set_sentiment(doc, y)

    def pipe(self, docs, batch_size=1000, n_threads=2):
        for doc in docs:
            Xs = get_features([doc], self.max_length)
            ys = self._model.predict(Xs)
            for doc, label in zip([doc], ys):
                doc.sentiment = label
            yield doc

    def set_sentiment(self, doc, y):
        doc.sentiment = float(y[0])


def get_labelled_sentences(docs, doc_labels):
    labels = []
    sentences = []
    for doc, y in zip(docs, doc_labels):
        sentences.append(doc)
        labels.append(y)
    return sentences, numpy.asarray(labels, dtype='int32')


def get_features(docs: list, max_length):
    Xs = numpy.zeros((len(docs), max_length), dtype='int32')
    for i, doc in enumerate(docs):
        j = 0
        for token in doc:
            vector_id = token.vocab.vectors.find(key=token.orth)
            if vector_id >= 0:
                Xs[i, j] = vector_id
            else:
                Xs[i, j] = 0
            j += 1
            if j >= max_length:
                break
    return Xs


def compile_lstm(embeddings, shape, settings):
    model = Sequential()
    model.add(
        Embedding(
            embeddings.shape[0],
            embeddings.shape[1],
            input_length=shape['max_length'],
            trainable=False,
            weights=[embeddings]
        )
    )
    model.add(TimeDistributed(Dense(shape['nr_hidden'], use_bias=False)))
    model.add(Bidirectional(LSTM(shape['nr_hidden'], recurrent_dropout=settings['dropout'], dropout=settings['dropout'])))
    model.add(Dense(shape['nr_class'], activation='relu'))
    model.add(Dropout(settings['dropout']))
    model.add(Dense(shape['nr_class'], activation='sigmoid'))
    model.compile(optimizer=Adam(lr=settings['lr']), loss='binary_crossentropy', metrics=['accuracy'])
    return model


def get_embeddings(vocab):
    return vocab.vectors.data


def train(train_texts, train_labels, dev_texts, dev_labels,
          lstm_shape, lstm_settings, lstm_optimizer, batch_size=100,
          nb_epoch=5, by_sentence=True):
    print("Loading spaCy")
    nlp = spacy.load(pt_br_model)
    embeddings = get_embeddings(nlp.vocab)
    model = compile_lstm(embeddings, lstm_shape, lstm_settings)

    print("Parsing texts...")
    train_docs = list(nlp.pipe(train_texts))
    dev_docs = list(nlp.pipe(dev_texts))
    print("Starting get_labelled_sentences()")
    if by_sentence:
        train_docs, train_labels = get_labelled_sentences(train_docs, train_labels)
        dev_docs, dev_labels = get_labelled_sentences(dev_docs, dev_labels)

    train_X = get_features(train_docs, lstm_shape['max_length'])
    dev_X = get_features(dev_docs, lstm_shape['max_length'])
    model.fit(train_X, train_labels, validation_data=(dev_X, dev_labels),
              epochs=nb_epoch, batch_size=batch_size)
    print("Model ready")
    return model


def evaluate(model_dir, texts, labels, max_length=100):
    nlp = spacy.load(pt_br_model)
    nlp.add_pipe(nlp.create_pipe('sentencizer'), first=True)
    nlp.add_pipe(SentimentAnalyser.load(model_dir, nlp, max_length=max_length))
    correct = 0
    i = 0
    for doc in nlp.pipe(texts, batch_size=1000, n_threads=4):
        if doc.sentiment >= 0.5 and bool(labels[i]) is True:
            print(texts[i] + " correctly classified as True " + str(doc.sentiment))
        elif doc.sentiment >= 0.5 and bool(labels[i]) is False:
            print(texts[i] + " wrongly classified as True " + str(doc.sentiment))
        elif doc.sentiment < 0.5 and bool(labels[i]) is True:
            print(texts[i] + " wrongly classified as False " + str(doc.sentiment))
        elif doc.sentiment < 0.5 and bool(labels[i]) is False:
            print(texts[i] + " correctly classified as False " + str(doc.sentiment))
        correct += bool(doc.sentiment >= 0.5) == bool(labels[i])
        i += 1
    return float(correct) / i


train_samples = [
    ('bom', 1),
    ('perfeito', 1),
    ('rápido', 1),
    ('ruim', 0),
    ('não', 0),
    ('melhorar', 0),
    ('demorado', 0),
    ('droga', 0)
]

validation_samples = [
    ('bom proposta', 1),
    ('servir perfeitamente', 1),
    ('não gostar', 0),
    ('produto ruim', 0)
]

test_samples = [
    ('bom', 1),
    ('empresa boa', 1),
    ('pode melhorar', 0),
    ('não gostei', 0)
]


def execute(model_dir=None, train_dir=None, dev_dir=None,
            is_runtime=False,
            nr_hidden=64, max_length=100, # Shape
            dropout=0.5, learn_rate=0.001, # General NN config
            nb_epoch=5, batch_size=256, nr_examples=-1):  # Training params
    train_texts = [item[0] for item in train_samples]
    train_labels = [item[1] for item in train_samples]
    val_texts = [item[0] for item in validation_samples]
    val_labels = [item[1] for item in validation_samples]
    
    
    if model_dir is not None:
        if not os.path.exists(model_dir):
          os.makedirs(model_dir)
        model_dir = pathlib.Path(model_dir)
    if is_runtime:
        test_texts = [item[0] for item in test_samples];
        test_labels = [item[1] for item in test_samples];
        acc = evaluate(model_dir, test_texts, test_labels, max_length=max_length)
        print(acc)
    else:
        print("Training neural network...")
        train_labels = numpy.asarray(train_labels, dtype='int')
        val_labels = numpy.asarray(val_labels, dtype='int')
        lstm = train(train_texts, train_labels, val_texts, val_labels,
                     {'nr_hidden': nr_hidden, 'max_length': max_length, 'nr_class': 1},
                     {'dropout': dropout, 'lr': learn_rate},
                     {}, nb_epoch=nb_epoch, batch_size=batch_size)
        weights = lstm.get_weights()
        if model_dir is not None:
            with (model_dir / 'model').open('wb') as file_:
                pickle.dump(weights[1:], file_)
            with (model_dir / 'config.json').open('w') as file_:
                file_.write(lstm.to_json())


def train_network():
	execute(model_dir='binary_classification', is_runtime=False, nb_epoch=25, nr_hidden=128)


def evaluate_network():
	execute(model_dir='binary_classification', is_runtime=True)

In [15]:
train_network()

Training neural network...
Loading spaCy
Parsing texts...
Starting get_labelled_sentences()
Train on 8 samples, validate on 4 samples
Epoch 1/25
8/8 [==============================] - 3s 417ms/step - loss: 0.7029 - acc: 0.6250 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/25
8/8 [==============================] - 0s 17ms/step - loss: 0.6805 - acc: 0.6250 - val_loss: 0.6929 - val_acc: 0.5000
Epoch 3/25
8/8 [==============================] - 0s 16ms/step - loss: 0.6568 - acc: 0.6250 - val_loss: 0.6991 - val_acc: 0.5000
Epoch 4/25
8/8 [==============================] - 0s 20ms/step - loss: 0.6420 - acc: 0.6250 - val_loss: 0.7062 - val_acc: 0.5000
Epoch 5/25
8/8 [==============================] - 0s 19ms/step - loss: 0.8554 - acc: 0.6250 - val_loss: 0.7056 - val_acc: 0.5000
Epoch 6/25
8/8 [==============================] - 0s 21ms/step - loss: 0.5875 - acc: 0.6250 - val_loss: 0.7093 - val_acc: 0.5000
Epoch 7/25
8/8 [==============================] - 0s 19ms/step - loss: 0.6595 - acc: 0.6250

In [16]:
evaluate_network()

bom wrongly classified as False 0.4780743420124054
empresa boa wrongly classified as False 0.4511517882347107
pode melhorar correctly classified as False 0.44887104630470276
não gostei correctly classified as False 0.4558592140674591
0.5
